# client instance 생성

In [23]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
DEPLOYMENT_URL="http://localhost:2024/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [24]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': 'de2393af-102b-4609-86c4-6e7aacf58b08', 'created_at': '2025-08-12T09:33:03.160668+00:00', 'updated_at': '2025-08-12T09:33:03.160669+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None}
de2393af-102b-4609-86c4-6e7aacf58b08


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [25]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [26]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [27]:
msg = """/add_design_prompt

{
"user_msg": "제주도 여행 슬라이드를 만들어주세요. 부가 설명없이 바로 만드세요."
    "design_prompt": "당신은 프레젠테이션 슬라이드 템플릿 디자이너입니다.
목표는 여러 장표와 다양한 레이아웃이 있어도, 전체 발표가 하나의 일관된 디자인으로 보이게 하는 것입니다.
아래 규칙을 항상 준수하여 슬라이드를 구성하세요.
 
[글꼴]
- 제목: 굵고 명확한 Sans-serif 폰트, 크기 32~40pt
- 본문: 가독성 좋은 Sans-serif 폰트, 크기 18~24pt
- 캡션/출처: 크기 12~14pt, 기울임체 가능
 
[색상]
- 기본 텍스트: #222222 (진한 회색)
- 배경: 흰색(#FFFFFF) 또는 매우 연한 톤(HEX: #F9F9F9 이하)
- 포인트 색상: 브랜드 컬러 1~2개만 사용
- 차트/그래프 색상: 포인트 색상과 중성색 혼합, 과도한 채도 피하기
- WCAG 대비 원칙 준수
 
[레이아웃]
- 각 슬라이드의 안전 여백 유지 (상하좌우 5% 이상)
- 제목 위치는 슬라이드 상단 고정
- 주요 콘텐츠는 그리드 또는 정렬 가이드에 맞춤
- 불필요한 장식 요소 최소화
- 슬라이드 당 하나의 핵심 메시지만 전달
 
[목록 / 블릿 스타일]
- 모든 블릿 포인트는 동일한 시각 언어로 유지
- 1단계: 꽉 찬 원형(•), 포인트 색상 100% 불투명
- 2단계: 작은 원형, 포인트 색상 70% 투명도
- 3단계: 아주 작은 사각형 또는 라인(▢), 포인트 색상 50% 투명도
- 블릿과 텍스트 사이 간격은 일정(0.4em)
- 계층 구조는 크기·색 농도·모양 변화로만 구분 (번호/문자 사용 금지)
- 줄 간격은 1.2~1.3배로 유지
 
[출처 / 저작권 표기]
- 이미지나 데이터 출처는 슬라이드 하단 오른쪽에 작게(12~14pt) 표기
- '출처:' 혹은 'Source:'로 시작
- 배경색과 대비되도록 흐린 회색(#888888) 또는 짙은 회색(#555555) 사용
 
[아이콘 / 이미지]
- 동일한 스타일 세트 사용 (선 두께, 채색 방식 통일)
- 이미지 모서리 처리 방식(둥근 모서리 or 직각) 일관성 유지
- 해상도 깨짐 방지, 비율 왜곡 금지
- 사진은 비율을 유지한 채로 전체가 보이도록 배치 (중요한 부분이 잘리지 않게)
- 필요 시 여백 또는 반투명 배경으로 보정하여 프레임에 맞춤
 
[템플릿 / 자동 일관성]
- Slide Master 또는 AI 자동 레이아웃 기능을 활용해 스타일 일괄 적용
- 로고, 브랜드 컬러, 폰트를 모든 슬라이드에 자동 반영
- 동일한 레이아웃 패턴을 유사 페이지에 반복 적용
 
[메시지 단순화]
- 슬라이드 당 하나의 키 메시지
- 텍스트 과부하 방지, 핵심 단어 중심 구성
- 불필요한 설명은 부록/백업 슬라이드로 분리
 
[시각 계층 & 대비]
- 제목 > 부제목 > 본문 계층적 배치
- 색상·폰트 크기·굵기를 활용한 시각적 계층 구조 명확화
- WCAG 대비 원칙을 고려해 텍스트·배경 대비 유지
 
[애니메이션 / 시각 효과]
- 강조 목적의 애니메이션만 사용 (예: Fade In)
- 동일 유형의 전환 효과만 사용
- 과도한 움직임과 시각적 소음 최소화
 
[전반 원칙]
- ‘간결함(Clarity)’과 ‘일관성(Consistency)’이 최우선
- 레이아웃이 달라도 여백, 색, 글꼴, 블릿 스타일, 출처 표기, 이미지 노출 방식은 동일하게 유지",
}
"""

In [28]:
input = { "messages": msg}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            print(chunk.data['agent']['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
##########  AI 메시지  ##########

##########  AI 메시지  ##########
##########  AI 메시지  ##########
##########  슬라이드 데이터  ##########
StreamPart(event='updates|workflow:eb720797-8b81-c13f-b222-3b7ccc1773e8|slide_generate_agent:85223ceb-81e9-cfdb-f135-9ce37220d839', data={'executor': {'slides': [{'topic': '제주도 여행 슬라이드', 'idx': 2, 'name': '제주도 여행 추천 명소', 'description': '제주도의 아름다운 여행 추천 명소에 대한 정보를 제공합니다. 명소의 특징, 위치, 즐길 거리 등을 포함해 짧은 소개와 여행 팁을 공유합니다.', 'research': True, 'content': '## 제주도 여행 추천 명소\n\n제주도는 한국의 보물 같은 섬으로, 수많은 아름다운 명소가 자리 잡고 있습니다. 가장 잘 알려진 곳 중 하나는 **성산 일출봉**입니다. 이곳은 일출을 보기 좋은 장소로, 일찍 일어나 해돋이를 감상하는 것을 추천합니다. 정상에 오르는 길은 다소 가파르지만, 정상에서 보는 전경은 그만한 가치가 있습니다[1].\n\n제주도의 중심에는 **한라산 국립공원**이 있습니다. 한라산은 남한 최고의 산으로, 다양한 등산 코스가 마련되어 있어 자연을 즐기기에 최적입니다. 특히 겨울에는 설산을 감상할 수 있어 색다른 경험이 될 것입니다[2].\n\n해안 명소로는 **함덕 해수욕장**이 유명합니다. 맑고 푸른 바닷물과 넓은 백사장이 있어 여름철 해수욕을 즐기기에 제격입니다. 또한, 해변 근처에는 많은 카페와 레스토랑이 있어서 휴식을 즐길 수 있습니다[3].\n\n**만장굴**은 제주도의 대표적인 동굴명소입니다. 이 동굴은 약 1만 년 전에 용암이 흐르면서 

In [5]:
input = { "messages": "안녕"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            print(chunk.data['agent']['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
안녕하세요! 슬라이드를 만드는 데 도움이 필요하신가요, 아니면 다른 질문이 있으신가요?


In [6]:
input = { "messages": "제주도 여행을 주제로 슬라이드를 만들어줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            data = get_data(chunk)
            print(data['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
좋아요, 제주도 여행 슬라이드를 만들기 위해 몇 가지 정보를 더 알고 싶습니다. 몇 가지 질문 드릴게요.

1. 슬라이드의 대상 청중은 누구인가요? 예를 들어, 친구들, 가족, 회사 동료 등.
2. 슬라이드에 포함하고 싶은 주요 내용은 무엇인가요? 예를 들어, 관광 명소, 음식 문화, 여행 팁 등.
3. 슬라이드의 개수를 알고 싶습니다.
4. 특별히 강조하고 싶은 주제가 있나요?


In [7]:
input = {"messages": "알아서 해줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):

    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        msg_id = data['messages'][-1]['id']

        print(data['messages'][-1]['content'])
    elif filter_silde_data(chunk):
        print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)


##########  AI 메시지  ##########

##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. Jeju Island Travel - This section will cover the most popular tourist destinations in Jeju, providing insights into must-visit sites, natural beauty, and unique attractions. 
3. Jeju Island Travel - This section will explore Jeju's local cuisine, highlighting traditional dishes, recommended restaurants, and unique culinary experiences specific to the island. 
4. Jeju Island Travel - This section provides guidance on planning trip itineraries in Jeju, including suggested tour routes that maximize sightseeing experiences over different durations, such as 2-day, 3-day trips. 
5. Jeju Island Travel - This section explores various accommodation choices available in Jeju, from budget to luxury, including hotels, guesthouses, and unique stays like traditional Korean houses. 
6. Jeju Island Travel - This section discusses the transportation sys

In [8]:
for chunk in list_chunk:
    if filter_silde_data(chunk):
        print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
        print(chunk.data)
        break

In [9]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [10]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

##########  AI 메시지  ##########
안녕하세요! 슬라이드를 만드는 데 도움이 필요하신가요, 아니면 다른 질문이 있으신가요?
##########  AI 메시지  ##########
좋아요, 제주도 여행 슬라이드를 만들기 위해 몇 가지 정보를 더 알고 싶습니다. 몇 가지 질문 드릴게요.

1. 슬라이드의 대상 청중은 누구인가요? 예를 들어, 친구들, 가족, 회사 동료 등.
2. 슬라이드에 포함하고 싶은 주요 내용은 무엇인가요? 예를 들어, 관광 명소, 음식 문화, 여행 팁 등.
3. 슬라이드의 개수를 알고 싶습니다.
4. 특별히 강조하고 싶은 주제가 있나요?
##########  AI 메시지  ##########

##########  AI 메시지  ##########
Successfully transferred to slide_create_agent
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. Jeju Island Travel - This section will cover the most popular tourist destinations in Jeju, providing insights into must-visit sites, natural beauty, and unique attractions. 
3. Jeju Island Travel - This section will explore Jeju's local cuisine, highlighting traditional dishes, recommended restaurants, and unique culinary experiences specific to the island. 
4. Jeju Island Travel - This section provides guidance on planning trip itineraries

In [11]:
len(state["values"]["slides"])

14

In [12]:
state["values"]["slides"]

[{'slide_type': 'title',
  'topic': '제주도 여행',
  'idx': 0,
  'name': '타이틀 슬라이드',
  'description': '제주도 여행에 대한 프레젠테이션 타이틀 페이지',
  'requirements': ['타이틀 텍스트', '부제목 또는 설명'],
  'research': True,
  'content': '## 타이틀 슬라이드\n\n제주도 여행 (Jeju Island Travel) Presentation\n\n**Overview:** \nJeju Island, often referred to as the "Hawaii of South Korea," stands as the country\'s largest volcanic island, recognized as a UNESCO World Natural Heritage site due to its unique geological features. Known for its captivating landscapes, the island offers diverse attractions such as picturesque beaches including Iho Tewoo Beach and Samyang Beach, the Olle Trails, and the renowned Hallasan Mountain, which is the tallest mountain in South Korea [1][2]. \n\n**제주도의 주요 특징:**\n\n- **언제 방문할까?**: 보통 봄과 여름이 제주도를 방문하기 좋은 시기입니다. 특히 봄의 끝자락과 여름은 제주도를 방문하기에 좋은 계절로 알려져 있습니다 [3].\n\n- **유네스코 세계자연유산**: 제주도는 독특한 화산 지형과 다양한 자연 경관 덕분에 유네스코 세계자연유산에 등재되었습니다. \n\n- **주요 관광지**: 우도, 한라산, 만장굴, 한라산 국립공원 및 용머리 해안과 같은 유명한 여행지가 사랑받고 있습니다 

In [13]:
for s in state["values"]["slides"]:
    print(s["idx"])

0
1
2
3
4
5
6
0
1
2
3
4
5
6


In [15]:
state["values"]

{'messages': [{'content': '안녕',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': 'e2f3e9a4-3f32-46ad-9c29-900e8673da58',
   'example': False},
  {'content': '안녕하세요! 슬라이드를 만드는 데 도움이 필요하신가요, 아니면 다른 질문이 있으신가요?',
   'additional_kwargs': {'refusal': None},
   'response_metadata': {'token_usage': {'completion_tokens': 27,
     'prompt_tokens': 118,
     'total_tokens': 145,
     'completion_tokens_details': {'accepted_prediction_tokens': 0,
      'audio_tokens': 0,
      'reasoning_tokens': 0,
      'rejected_prediction_tokens': 0},
     'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
    'model_name': 'gpt-4o-2024-08-06',
    'system_fingerprint': 'fp_ee1d74bde0',
    'id': 'chatcmpl-C3eEolQ7WujJ5D9Ex345Oc841KpN7',
    'service_tier': None,
    'finish_reason': 'stop',
    'logprobs': None,
    'content_filter_results': {}},
   'type': 'ai',
   'name': 'supervisor',
   'id': 'run--b3776c3c-8c9b-40ba-99d8-3c5f32814e3c-0'